In [2]:
pip install opencv-python opencv-python-headless numpy

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install secure-smtplib

Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import numpy as np
import time
import smtplib
from datetime import datetime
from ultralytics import YOLO
from cv2 import VideoCapture

In [7]:
import cv2
import numpy as np
import time
import smtplib
from datetime import datetime
from ultralytics import YOLO
from IPython.display import display, Image, clear_output

# Load YOLOv8 model
model = YOLO('yolov8n.pt') 

# Initialize the video capture
cap = cv2.VideoCapture(0)  # Use '0' for the webcam

def send_alert_email():
    sender_email = "surveillance.user.alert@gmail.com"
    receiver_email = "dhruv0001638@gmail.com" #change reciever emailid or link to a database
    password = "ouju rhrx xnss nipg"
    message = "Subject: Alert\n\nPerson detected after 1 AM."

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
        server.quit()
        print("Alert email sent.")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Flag to stop processing once a person is detected and an email is sent
person_detected_and_alert_sent = False

while not person_detected_and_alert_sent:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Detecting objects
    results = model(frame)

    # Process results
    boxes = results[0].boxes.xyxy.numpy()  # Bounding box coordinates
    confidences = results[0].boxes.conf.numpy()  # Confidence scores
    class_ids = results[0].boxes.cls.numpy()  # Class IDs

    for i, box in enumerate(boxes):
        if confidences[i] > 0.5 and class_ids[i] == 0:  # '0' is the class ID for 'person' in YOLOv8
            x1, y1, x2, y2 = map(int, box)
            label = 'person'
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Check the current time
            current_time = datetime.now().time()
            print(f"Current time: {current_time}")
            if 1 <= current_time.hour < 6:  # After 1 AM
                print("Person detected after 1 AM.")
                send_alert_email()
                person_detected_and_alert_sent = True
                break  # Exit the inner for loop

    if person_detected_and_alert_sent:
        break  # Exit the while loop

    # Display the frame in Jupyter Notebook
    _, buffer = cv2.imencode('.jpg', frame)
    frame_image = buffer.tobytes()
    display(Image(data=frame_image))
    clear_output(wait=True)

cap.release()
cv2.destroyAllWindows()
print("Video capture stopped.")

KeyboardInterrupt: 